# Explore Query History

In [ ]:
import os
from pathlib import Path

from dotenv import find_dotenv, load_dotenv
from sqlalchemy import create_engine
from snowflake.sqlalchemy import URL

In [ ]:
PROJ_ROOT = Path().resolve().parents[3]
env_file_dir = PROJ_ROOT / '.env'
_ = load_dotenv(env_file_dir, verbose=True)

## About

Show query history and view executions of granting the `reporting` role

1. `usage` privilige
2. read-only (`select`) access

### Notes

1. This notebook supports <kbd>Run</kbd> > <kbd>Run All Cells</kbd>.

## User Inputs

In [ ]:
#

In [ ]:
engine = create_engine(
    URL(
        drivername="driver",
        account=os.getenv("UPLIMIT_SNOWFLAKE_ACCOUNT"),
        user=os.getenv("UPLIMIT_SNOWFLAKE_USER"),
        password=os.getenv("UPLIMIT_SNOWFLAKE_PASS"),
        warehouse=os.getenv("UPLIMIT_SNOWFLAKE_WAREHOUSE"),
        role=os.getenv("UPLIMIT_SNOWFLAKE_ROLE"),
        database=os.getenv("UPLIMIT_SNOWFLAKE_DB_NAME"),
        schema=os.getenv("UPLIMIT_SNOWFLAKE_SCHEMA"),
        timezone='US/Eastern'
    )
)

## Connect

Load Jupyter SQL extension

In [ ]:
%load_ext sql

Set the maximum number of rows to be displayed to `None` (shows all rows)

In [ ]:
%config SqlMagic.displaylimit = None

Connect to DuckDB database

In [ ]:
%sql engine --alias connection

## Queries

### Schema Granted `USAGE` Privilige

Show all schemas to which the `reporting` role was successfully granted `USAGE` privilege

In [ ]:
%%sql
SELECT query_id,
       query_text,
       SPLIT_PART(SPLIT(query_text, ' ON SCHEMA')[1], ' TO ROLE', 1) AS schema_name,
       SPLIT_PART(RTRIM(query_text, ';'), ' TO ROLE ', 2) AS role_granted,
       start_time,
       rows_produced AS num_rows,
       execution_status AS status,
       role_name
FROM table(
    information_schema.query_history(
        END_TIME_RANGE_START=>to_timestamp_ltz('2024-11-1 21:00:00.000 -0400'),
        RESULT_LIMIT=>500
    )
)
WHERE query_text LIKE 'GRANT USAGE%'
ORDER BY start_time DESC

**Notes**

1. There are multiple `GRANT USAGE`s since that command was run multiple times.

Count the number of models to which the `reporting` role was successfully granted read-only (`SELECT`) access

In [ ]:
%%sql
/* count number of models to which the reporting role was granted read-only
access */
SELECT SPLIT_PART(
           SPLIT(query_text, ' ON SCHEMA')[1], ' TO ROLE', 1
       ) AS schema_name,
       SPLIT_PART(RTRIM(query_text, ';'), ' TO ROLE ', 2) AS role_granted,
       COUNT(DISTINCT(query_text)) AS num_usage_grants
FROM table(
    information_schema.query_history(
        END_TIME_RANGE_START=>to_timestamp_ltz('2024-11-1 21:00:00.000 -0400'),
        RESULT_LIMIT=>500
    )
)
-- get GRANT SELECts
WHERE query_text LIKE 'GRANT USAGE%'
-- get successful GRANts
AND execution_status = 'SUCCESS'
GROUP BY ALL

### Models Granted Read-Only (`SELECT`) Access

Show all models to which the `reporting` role was successfully granted read-only (`SELECT`) access

In [ ]:
%%sql
SELECT query_id,
       query_text,
       SPLIT_PART(
           ARRAY_SLICE(SPLIT(query_text, ' TO ROLE'), 0, 1)[0], '.', 3
       ) AS model_name,
       SPLIT_PART(RTRIM(query_text, ';'), ' TO ROLE ', 2) AS role_granted,
       start_time,
       rows_produced AS num_rows,
       execution_status AS status,
       role_name
FROM table(
    information_schema.query_history(
        END_TIME_RANGE_START=>to_timestamp_ltz('2024-11-1 21:00:00.000 -0400'),
        RESULT_LIMIT=>500
    )
)
WHERE query_text LIKE 'GRANT SELECT%'
ORDER BY start_time DESC

Count the number of models to which the `reporting` role was successfully granted read-only (`SELECT`) access

In [ ]:
%%sql
/* count number of models to which the reporting role was granted read-only
access */
SELECT COUNT(DISTINCT(query_text)) AS num_models_granted_access
FROM table(
    information_schema.query_history(
        END_TIME_RANGE_START=>to_timestamp_ltz('2024-11-1 21:00:00.000 -0400'),
        RESULT_LIMIT=>500
    )
)
-- get GRANT SELECts
WHERE query_text LIKE 'GRANT SELECT%'
-- get successful GRANts
AND execution_status = 'SUCCESS'

## Disconnect

Close connection

In [ ]:
%sql --close connection